In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile

# TensorFlow for the model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Scikit-learn for data handling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# --- Unzip the dataset ---
ZIP_FILE = 'landmark_subset.zip'
EXTRACT_DIR = 'image_subset'

if not os.path.exists(EXTRACT_DIR):
    print(f"Extracting {ZIP_FILE}...")
    with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("Extraction complete! ✅")
else:
    print(f"Directory '{EXTRACT_DIR}' already exists.")

Directory 'image_subset' already exists.


In [6]:
# --- Load the full training metadata ---
# After
# After
# After
df = pd.read_csv(
    'train.csv',
    sep=',',
    encoding='latin-1',
    engine='python',
    quoting=3
)

# --- Filter the dataframe for your downloaded images ---
image_files = os.listdir(EXTRACT_DIR)
image_ids = [file.split('.')[0] for file in image_files]

df_subset = df[df['id'].isin(image_ids)].copy()

# Create the full file path for each image
df_subset['filepath'] = df_subset['id'].apply(lambda x: os.path.join(EXTRACT_DIR, f'{x}.jpg'))

# --- Encode the landmark_id labels ---
label_encoder = LabelEncoder()
df_subset['label'] = label_encoder.fit_transform(df_subset['landmark_id'])

N_CLASSES = len(label_encoder.classes_)
print(f"Working with {len(df_subset)} images from {N_CLASSES} landmark classes.")

# --- Split data into training and validation sets ---
X = df_subset['filepath']
y = df_subset['label']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Working with 18179 images from 10 landmark classes.


In [7]:
IMG_SIZE = 224 # Input size for EfficientNet
BATCH_SIZE = 32

# Create dataframes for the generator
train_df = pd.DataFrame({'filepath': X_train, 'label': y_train.astype(str)})
val_df = pd.DataFrame({'filepath': X_val, 'label': y_val.astype(str)})

# Data generator for training with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Data generator for validation (only rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 14543 validated image filenames belonging to 10 classes.
Found 3636 validated image filenames belonging to 10 classes.


In [8]:
from tensorflow.keras.layers import Input

# --- Manually define a 3-channel input layer ---
# This forces the model to accept the correct shape.
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# --- Load the base model, connecting it to our manual input layer ---
# We use 'input_tensor' instead of 'input_shape' to perform the override.
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_tensor=inputs
)
base_model.trainable = False # Freeze the base layers

# --- Add our custom layers ---
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(N_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# --- Compile the model ---
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

ValueError: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (3, 3, 3, 32)